In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../scraping/cleansed_data/players_20230425190937.csv', sep=';')

# Erstelle eine Kopie des Dataframes
df_copy = df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18335 entries, 0 to 18334
Data columns (total 48 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             18335 non-null  object 
 1   League                    18238 non-null  object 
 2   NationalLeagueLevel       18335 non-null  int64  
 3   Club                      18331 non-null  object 
 4   No                        17905 non-null  float64
 5   Name                      18331 non-null  object 
 6   Value                     17899 non-null  float64
 7   ValueLastUpdate           17899 non-null  object 
 8   HighestValue              18004 non-null  float64
 9   HighestValueDate          18004 non-null  object 
 10  ValueCurrency             17899 non-null  object 
 11  Age                       18328 non-null  float64
 12  Height                    17052 non-null  float64
 13  Nationality               18331 non-null  object 
 14  Foot  

## Entfernen von Spalten und Zeilen

In [3]:
# Entferne Source und ScrapingTime
df = df.drop(['Source', 'ScrapingTime'], axis=1)

In [4]:
# Entferne ValueCurrency da immer Euro
df = df.drop(['ValueCurrency'], axis=1)

In [5]:
# Lösche alle Spieler ohne Value
df = df[df['Value'].notnull()]

## Datentypen anpassen

In [6]:
# 'ClubSince' und 'ContractUntil' als Datumsangaben einlesen
df['ClubSince'] = pd.to_datetime(df['ClubSince'])
df['ContractUntil'] = pd.to_datetime(df['ContractUntil'])

In [7]:
# 'ClubSince' und 'ContractUntil' als float Werte umwandeln (Differenz in Tagen ab Scrapingtag 11.04.2023, da dann auch die Values ermittelt wurden)
df['ClubSince'] = ((pd.to_datetime('2023-04-11') - df['ClubSince']) / np.timedelta64(1, 'D')).astype(float)
df['ContractUntil'] = ((df['ContractUntil'] - pd.to_datetime('2023-04-11')) / np.timedelta64(1, 'D')).astype(float)

## Entferne Dublikate

In [8]:
# Doppelte Spieler aus dem Dataframe entfernen
df = df.drop_duplicates(subset=['Name', 'Age', 'Nationality', 'Club'], keep='first')

## Fehlende Werte ersetzen (Objekte und logische Werte)

In [9]:
# Ersetze NaN-Werte mit 'Unbekannt'
df['Nationality'].fillna('unbekannt', inplace=True)
df['Foot'].fillna('unbekannt', inplace=True)
df['Position'].fillna('unbekannt', inplace=True)
df['Supplier'].fillna('unbekannt', inplace=True)
df['Consultancy'].fillna('unbekannt', inplace=True)

In [10]:
# Erstelle ein Dictionary mit den Positionen und ihren entsprechenden Kategorien
pos_dict = {'Torwart': 'Torwart',
            'Innenverteidiger': 'Abwehr',
            'Linker Verteidiger': 'Abwehr',
            'Rechter Verteidiger': 'Abwehr',
            'Defensives Mittelfeld': 'Mittelfeld',
            'Zentrales Mittelfeld': 'Mittelfeld',
            'Linkes Mittelfeld': 'Mittelfeld',
            'Offensives Mittelfeld': 'Mittelfeld',
            'Linksaußen': 'Sturm',
            'Rechtsaußen': 'Sturm',
            'Mittelstürmer': 'Sturm',
            'Rechtes Mittelfeld': 'Mittelfeld',
            'Hängende Spitze': 'Mittelfeld',
            'Mittelfeld': 'Mittelfeld',
            'Abwehr': 'Abwehr',
            'Sturm': 'Sturm'}

# Erstelle eine neue Spalte mit der Positionskategorie
df['PositionCategory'] = df['Position'].map(pos_dict)

In [11]:
df['InternationalTeam'].fillna('kein_internationales_Team', inplace=True)

In [12]:
df.loc[df['InternationalTeam'] == 'kein_internationales_Team', ['InternationalGames', 'InternationalGoals']] = 0

In [13]:
df['Instagram'].fillna('kein_instagram', inplace=True)

In [14]:
# Setze die Torwart-spezifischen Werte auf 0, wenn die Position des Spielers keine Torwart-Position ist und der jeweilige Wert NaN ist
df.loc[(df['Position'] != 'Torwart') & (df['TwPenaltySavesQuote'].isna()), 'TwPenaltySavesQuote'] = 0
df.loc[(df['Position'] != 'Torwart') & (df['TwGoalsConceded'].isna()), 'TwGoalsConceded'] = 0
df.loc[(df['Position'] != 'Torwart') & (df['TwCleanSheets'].isna()), 'TwCleanSheets'] = 0

In [15]:
# Setze die Werte auf 0, wenn die Position des Spielers nicht im Sturm ist
df.loc[df['PositionCategory'] != 'Sturm', ['FsAssists', 'FsPenaltyGoals', 'FsMinutesPerGoal', 'FsGoalParticipationQuote']] = 0

In [16]:
# Ein Wörterbuch erstellen, um den Verein und die entsprechende Liga zu speichern
club_league_dict = df[df['League'].notnull()][['Club', 'League']].set_index('Club').to_dict()['League']

# Eine Funktion definieren, um fehlende Ligawerte anhand des Vereinsnamens zu ersetzen
def impute_league(row):
    if pd.isnull(row['League']) and row['Club'] in club_league_dict:
        return club_league_dict[row['Club']]
    else:
        return row['League']

# Die Funktion anwenden, um fehlende Werte zu ersetzen
df['League'] = df.apply(impute_league, axis=1)

In [17]:
# Gruppieren Sie das DataFrame nach Position und berechnen Sie den Modus der Spieler-Nummern für jede Position
mode_no_by_position = df.groupby('Position')['No'].agg(lambda x: x.mode()[0])

# Funktion zum Imputieren von fehlenden Spieler-Nummern (No) basierend auf der Position
def impute_no_by_position(row):
    if pd.isnull(row['No']):
        return mode_no_by_position[row['Position']]
    else:
        return row['No']

# Anwenden der Funktion auf das gesamte DataFrame, um fehlende Spieler-Nummern zu imputieren
df['No'] = df.apply(impute_no_by_position, axis=1)

In [18]:
# Setze NaN-Werte in Karten-Werten auf 0
df['YellowCards'].fillna(0, inplace=True)
df['YellowRedCards'].fillna(0, inplace=True)
df['RedCards'].fillna(0, inplace=True)

In [19]:
# Objekte haben keine fehlenden Werte mehr
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17803 entries, 0 to 18334
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             17803 non-null  object 
 1   League                    17803 non-null  object 
 2   NationalLeagueLevel       17803 non-null  int64  
 3   Club                      17803 non-null  object 
 4   No                        17803 non-null  float64
 5   Name                      17803 non-null  object 
 6   Value                     17803 non-null  float64
 7   ValueLastUpdate           17803 non-null  object 
 8   HighestValue              17803 non-null  float64
 9   HighestValueDate          17803 non-null  object 
 10  Age                       17800 non-null  float64
 11  Height                    16713 non-null  float64
 12  Nationality               17803 non-null  object 
 13  Foot                      17803 non-null  object 
 14  Positi

In [20]:
# Fehlende Werte in Prozent pro Variable
print(df.isnull().sum() / len(df) * 100)

LeagueCountry                0.000000
League                       0.000000
NationalLeagueLevel          0.000000
Club                         0.000000
No                           0.000000
Name                         0.000000
Value                        0.000000
ValueLastUpdate              0.000000
HighestValue                 0.000000
HighestValueDate             0.000000
Age                          0.016851
Height                       6.122564
Nationality                  0.000000
Foot                         0.000000
Position                     0.000000
Consultancy                  0.000000
Supplier                     0.000000
ClubSince                    0.196596
ContractUntil                6.027074
Injury                       0.000000
InternationalTeam            0.000000
ActiveInternational          0.000000
FormerInternational          0.000000
InternationalGames           0.000000
InternationalGoals           0.000000
StartingElevenQuote         11.402573
MinutesQuote

In [21]:
df.to_csv('../data/df_clean.csv', index=False)

## Nicht verwendeter Code

In [ ]:
null_values = df['Minutes'].isnull().groupby([df['LeagueCountry'], df['League']]).sum().to_frame()
size_values = df.groupby(['LeagueCountry', 'League']).size()

null_values['Players'] = size_values
null_values = null_values.rename(columns={'Minutes': 'NullValues'})
null_values['NullValuesRelative'] = null_values['NullValues'] / null_values['Players'] * 100

null_values = null_values[null_values['Players'] > 20]  # exclude minor leagues
null_values = null_values.sort_values('NullValuesRelative', ascending=False)

print(null_values)